In [2]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.5.0-rc3
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


In [3]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

In [6]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)


In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [9]:
history = model.fit(
    train_data.shuffle(10000).batch(512),
    epochs=10,
    validation_data= validation_data.batch(512),
    verbose=1
)

Epoch 1/10
30/30 [==============================] - 13s 420ms/step - loss: 0.6406 - accuracy: 0.5506 - val_loss: 0.5857 - val_accuracy: 0.6155
Epoch 2/10
30/30 [==============================] - 12s 410ms/step - loss: 0.5144 - accuracy: 0.7238 - val_loss: 0.4789 - val_accuracy: 0.7519
Epoch 3/10
30/30 [==============================] - 12s 412ms/step - loss: 0.3855 - accuracy: 0.8353 - val_loss: 0.3927 - val_accuracy: 0.8127
Epoch 4/10
30/30 [==============================] - 12s 411ms/step - loss: 0.2808 - accuracy: 0.8912 - val_loss: 0.3423 - val_accuracy: 0.8521
Epoch 5/10
30/30 [==============================] - 12s 412ms/step - loss: 0.2024 - accuracy: 0.9293 - val_loss: 0.3191 - val_accuracy: 0.8643
Epoch 6/10
30/30 [==============================] - 12s 413ms/step - loss: 0.1472 - accuracy: 0.9549 - val_loss: 0.3145 - val_accuracy: 0.8565
Epoch 7/10
30/30 [==============================] - 12s 412ms/step - loss: 0.1083 - accuracy: 0.9703 - val_loss: 0.3099 - val_accuracy: 0.8686

In [10]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 1s - loss: 0.3633 - accuracy: 0.8520
loss: 0.363
accuracy: 0.852
